<div class="alert alert-warning">
    
<b>Disclaimer:</b> The main objective of the <i>Jupyter</i> notebooks is to show how to use the models of the <i>QENS library</i> by
    
- building a fitting model: composition of models, convolution with a resolution function
- setting and run the fit   
- extracting and displaying information about the results

These steps have a minimizer-dependent syntax. That's one of the reasons why different minimizers have been used in the notebooks provided as examples.  
But, the initial guessed parameters might not be optimal, resulting in a poor fit of the reference data.
</div>


# Example: Equivalent sites circle model fitted with lmfit

## Table of Contents

- [Introduction](#Introduction)
- [Importing the required libraries](#Importing-the-required-libraries)
- [Plot of the fitting model](#Plot-of-the-fitting-model)
- [Creating the reference data](#Creating-the-reference-data)
- [Setting and fitting](#Setting-and-fitting)  
- [Plotting the results](#Plotting-the-results)  

[Top](#Table-of-Contents)

## Introduction

<div class="alert alert-info">
    
The objective of this notebook is to show how to use the <b>Equivalent Sites Circle</b> model to perform some 
fits using <a href="https://lmfit.github.io/lmfit-py/">lmfit</a>.
</div>

### Physical units
Please note that the following units are used for the QENS models

| Type of parameter | Unit          |
| :---------------- | :-----------: |
| Time              | picosecond    |
| Length            | Angstrom      |
| Momentum transfer | 1/Angstrom    |

For information about unit conversion, please refer to the jupyter notebook called `Convert_units.ipynb` in the `tools` folder.

The dictionary of units defined in the cell below specify the units of the refined parameters adapted to the convention used in the experimental datafile.

In [ ]:
# Units of parameters for selected QENS model and experimental data
dict_physical_units = {'omega': "1/ps", 
                       'q': "1/Angstrom", 
                       'scale': "unit_of_signal/ps", 
                       'center': "1/ps", 
                       'radius': "Angstrom", 
                       'resTime': "ps"}

[Top](#Table-of-Contents)

## Importing the required librairies

In [ ]:
# import python modules for plotting, fitting
from __future__ import print_function
import numpy as np

# for interactivity (plots, buttons...)
from pandas import DataFrame
import panel.widgets as pnw

import panel as pn
pn.extension()

import matplotlib.pyplot as plt
# %matplotlib notebook

In [ ]:
# install QENSmodels (if not already installed)
import pkgutil
import sys

if not pkgutil.find_loader("QENSmodels"):
    buttonY = pnw.Button(name='Yes', button_type='success')
    buttonN = pnw.Button(name='No', button_type='danger')
    choice_installation = panel.Column("Do you want to install the QENSmodels' library?", panel.Row(buttonY, buttonN))
    display(choice_installation)

In [ ]:
if not pkgutil.find_loader("QENSmodels"):
    if buttonY.clicks>0:
        !{sys.executable} -m pip install git+https://github.com/QENSlibrary/QENSmodels#egg=QENSmodels
    elif buttonN.clicks>0:
        print("You will not be able to run some of the remaining parts of this notebook")

In [ ]:
# install lmfit (if not already installed)
if not pkgutil.find_loader("lmfit"):
    lmfitY = pnw.Button(name='Yes', button_type='success')
    lmfitN = pnw.Button(name='No', button_type='danger')
    choice_installation = pn.Column("Do you want to install lmfit?", pn.Row(lmfitY, lmfitN))
    display(choice_installation)

In [ ]:
if not pkgutil.find_loader("lmfit"):
    if lmfitY.clicks>0:
         !{sys.executable} -m pip install lmfit
    elif lmfitN.clicks>0:
        print("You will not be able to run some of the remaining parts of this notebook")

In [ ]:
# required imports from lmfit
from lmfit import Model, Parameters

# import model from QENS library
import QENSmodels

[Top](#Table-of-Contents)

## Plot of the fitting model

The widget below shows the peak shape function imported from QENSmodels where the function's parameters can be varied.

In [ ]:
# Dictionary of initial values
ini_values = {'q': 1., 'scale': 5., 'center': 5.,  'Nsites': 3, 'radius': 5., 'resTime': 1.} 

# Define function to plot
def mplplot(df, **kwargs):
    fig = df.plot(legend=False).get_figure()
    plt.grid()
    plt.ylabel('Equivalent Sites Circle')
    plt.xlabel('x')
    plt.close(fig)
    return fig

def equiv_sites_circle(q=1.0, scale=1.0, center=0.0, Nsites=3, radius=1.0, resTime=1.0, view_fn=mplplot):
    xs = np.linspace(-10,10,100)
    ys = QENSmodels.sqwEquivalentSitesCircle(xs, q, scale, center, Nsites, radius, resTime) 
    df = DataFrame(dict(y=ys), index=xs)
    return view_fn(df, q=q, scale=scale, center=center, Nsites=Nsites, radius=radius, resTime=resTime)

# Define sliders and actions on plot
slider_q = pnw.FloatSlider(name='q', value=ini_values['q'], start=0.2, end=10)
slider_scale  = pnw.FloatSlider(name='scale', value=ini_values['scale'], start=1., end=10.)
slider_center = pnw.FloatSlider(name='center', value=ini_values['center'], start=0., end=10.)
slider_Nsites = pnw.IntSlider(name='Nsites', value=ini_values['Nsites'], start=2, end=20)
slider_radius = pnw.FloatSlider(name='radius', value=ini_values['radius'], start=0., end=10.)
slider_resTime =  pnw.FloatSlider(name='resTime', value=ini_values['resTime'], start=1., end=10.)

def update(event):
    site_equiv_panel[0] = equiv_sites_circle(slider_q.value, slider_scale.value, slider_center.value, slider_Nsites.value, slider_radius.value, slider_resTime.value)
    
slider_q.param.watch(update, 'value')
slider_scale.param.watch(update, 'value')
slider_center.param.watch(update, 'value')
slider_Nsites.param.watch(update, 'value')
slider_radius.param.watch(update, 'value')
slider_resTime.param.watch(update, 'value')

# Define reset button
reset_button = pnw.Button(name='Reset')

def on_click(event):
    """Reset the interactive plots to inital values."""
    slider_q.value = ini_values['q']
    slider_scale.value = ini_values['scale']
    slider_center.value = ini_values['center']
    slider_Nsites.value = ini_values['Nsites']
    slider_radius.value = ini_values['radius']
    slider_resTime.value = ini_values['resTime']
    
reset_button.param.watch(on_click, 'clicks')

# Define layout: title, plot, sliders and reset button
widgets = pn.Column("#### Equivalent Sites Circle", slider_q, slider_scale, slider_center, slider_Nsites, slider_radius, slider_resTime, reset_button)
site_equiv_panel = pn.Row(equiv_sites_circle(slider_q.value, slider_scale.value, slider_center.value, slider_Nsites.value, slider_radius.value, slider_resTime.value), widgets)

site_equiv_panel

[Top](#Table-of-Contents)

## Creating the reference data

In [ ]:
xx = np.linspace(-5,5,200)
equiv_sites_circle_noisy = QENSmodels.sqwEquivalentSitesCircle(xx, 1., 1.3, 0.3, 5, 4., 3.)*(1+0.1*np.random.normal(0,1,200)) + 0.01*np.random.normal(0, 1, 200) 

[Top](#Table-of-Contents)

## Setting and fitting

In [ ]:
gmodel = Model(QENSmodels.sqwEquivalentSitesCircle)

print('Names of parameters:', gmodel.param_names)
print('Independent variable(s):', gmodel.independent_vars)

#initial_parameters_values = [1.22, 0.25, 5, 0.3, 0.33]
# initial_parameters_values = [1.22, 0.2, 5, 3.1, 2]
ini_values = {'scale': 1.22, 'center': 0.2, 'Nsites': 5, 'radius': 3.1, 'resTime': 0.33}

# Define boundaries for parameters to be refined
gmodel.set_param_hint('scale', min=0)
gmodel.set_param_hint('center', min=-5, max=5)
gmodel.set_param_hint('radius', min=0)
gmodel.set_param_hint('resTime', min=0)

# Fix some of the parameters
gmodel.set_param_hint('q', vary=False)
gmodel.set_param_hint('Nsites', vary=False)

# Fit
result = gmodel.fit(equiv_sites_circle_noisy, w=xx, q=1.,
                    scale=ini_values['scale'], 
                    center=ini_values['center'],
                    Nsites=ini_values['Nsites'],
                    radius=ini_values['radius'],
                    resTime=ini_values['resTime'])

In [ ]:
# Plots - Initial model and reference data
fig0, ax0 = plt.subplots()
ax0.plot(xx, equiv_sites_circle_noisy, 'b.-', label='reference data')
ax0.plot(xx, result.init_fit, 'k--', label='model with initial guesses')
ax0.set(xlabel='x', title='Initial model and reference data')
ax0.grid()
ax0.legend();

[Top](#Table-of-Contents)

## Plotting results

using methods implemented in `lmfit`

In [ ]:
# display result
print('Result of fit:\n',result.fit_report())

# plot fitting result using lmfit functionality
result.plot()

In [ ]:
# plot fitting resulting using matplotlib.pyplot
fig1, ax1 = plt.subplots()
ax1.plot(xx, equiv_sites_circle_noisy, 'b-', label='reference data')
ax1.plot(xx, result.best_fit, 'r', label='fitting result')
ax1.legend()
ax1.set(xlabel='x', title='Fit result and reference data')
ax1.grid();

In [ ]:
for item in ['resTime', 'radius', 'center', 'scale']:
    print(item, result.params[item].value, '+/-', result.params[item].stderr, dict_physical_units[item])